<a href="https://colab.research.google.com/github/Jayveersinh-Raj/intro_NLP_tensorflow/blob/main/Intro_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to NLP funtamentals in TensorFlow

* NLP is used to derive information form natural language (Sequences, texts, speeches.

* Another common terms for NLP problems is Sequence to Sequence problems (seq2seq)

In [1]:
# We will need GPU to work with deep learning tasks such as NLP
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-678d0afe-3c4a-9db9-0e11-a4311bb2cf0b)


In [2]:
# For more detailed info use without -L
!nvidia-smi

Sat Aug 13 14:58:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Lets import our helper functions
!wget https://raw.githubusercontent.com/Jayveersinh-Raj/helper_functions/main/helper_functions.py
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-08-13 14:58:10--  https://raw.githubusercontent.com/Jayveersinh-Raj/helper_functions/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-08-13 14:58:11 (95.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



# Importing the dataset
Lets use Kaggle's Intro to NLP dataset

It is the text sample of tweets lablled as disasters or not disasters


In [4]:
# Lets get the data from google storage where Daniel has stored it
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data("nlp_getting_started.zip") # This is the name of the zip file that Daniel stored on google

--2022-08-13 14:58:13--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-08-13 14:58:14 (146 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



# Visualisation of the data
There are multiple ways:
* Python libraries to read and write files (refer [realpython](https://realpython.com)
* Pandas (But it would take up ram in colab)
* Using tensorflow to load text


In [5]:
import pandas as pd
data_train = pd.read_csv("train.csv")
data_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# Lets shuffle the data using pandas method
shuffled_train = data_train.sample(frac = 1, random_state = 42) # frac = 1 means 100% of the data shuffle
shuffled_train.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# Lets see how test looks like
data_test = pd.read_csv("test.csv")
data_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# Lets shuffle this too
shuffled_test = data_test.sample(frac = 1, random_state = 42) # random state is always use while doing some random activity
shuffled_test.head()

,id,keyword,location,text
2406,8051,refugees,NaN,Refugees as citizens - The Hindu http://t.co/G...
134,425,apocalypse,Currently Somewhere On Earth,@5SOStag honestly he could say an apocalypse i...
411,1330,blown%20up,Scout Team,If you bored as shit don't nobody fuck wit you...
203,663,attack,NaN,@RealTwanBrown Yesterday I Had A Heat Attack ?...
889,2930,danger,Leeds,The Devil Wears Prada is still one of my favou...


In [10]:
# Lets see the value counts to see how many examples we have of each category
shuffled_train.target.value_counts() # Because target contains our categories

0    4342
1    3271
Name: target, dtype: int64

In [16]:
# Lets visualize some random data to get the idea
import random
rand = random.randint(0, len(data_train)-3) # Not to exceed the total length
for i in shuffled_train[["text", "target"]][rand : rand+5 ].itertuples(): # because it will return dataframe as tuples
      _, text, target = i
      print(f"Target : {target}", "real disaster" if target > 0 else "not a diaster")
      print(f"Text : {text} \n")
      print("--- \n")
      

Target : 0 not a diaster
Text : @jaureguiswisdom lmao well i only know one and ive only had a crush on this one sooo 

--- 

Target : 0 not a diaster
Text : @Hollyorange8 my day has been a disaster of emotions 

--- 

Target : 0 not a diaster
Text : Aw man. 'Apollo Crews' just screams 'we can't think of a name for this black guy quick name some and mash them together' 

--- 

Target : 0 not a diaster
Text : @Bonn1eGreer The Angel of History propelled into the future by the winds of progress leaves in its wake piles of death and destruction. WB 

--- 

Target : 0 not a diaster
Text : :: Survive?? 

--- 

